In [2]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import numpy as np
from pandas import DataFrame,Series
from matplotlib.colors import ListedColormap
from random import sample
import matplotlib.pyplot as plt


In [28]:
# Data of 15 people for training & testing the model, splitting the train-test set 
df_features = pd.read_csv("H:/mastersProject/activity_analyzer/LogisticRegression/Data/featuresfile_10.csv")
df_features_3people = pd.read_csv("H:/mastersProject/activity_analyzer/LogisticRegression/Data/featuresfile.csv")

frames = [df_features, df_features_3people]
df_15 = pd.concat(frames)

#Drop duplicates
df_unique = df_15.drop_duplicates(subset=['User', 'Timestamp'])
df_unique.head()
df_unique.describe()
print("Shape of training and testing data", df_unique.shape)

X_data = df_unique.values[:, 2:45]
y_data = df_unique.values[:, 45]
usersList = set(df_unique.values[:,0])
print(len(usersList)+2) # Userid is for 3 people hence  

Shape of training and testing data (821, 46)
15


In [33]:
# Splitting the training and testing set by 33%
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size = 0.33, random_state = 42)
# Fitting the logistic regression model
clf = LogisticRegression(C=0.01, random_state=1)
clf.fit(X_train_data, y_train_data)

LogisticRegression(C=0.01, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=1, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [34]:
predict = clf.predict(X_test)
logisticRegScore = clf.score(X_test, y_test)

plt.figure(1, figsize=(4, 3))
plt.clf()
print("Logistic regression Score")
print(logisticRegScore*100)
print("Coefficients of the features")
print(clf.coef_)
print(X_train_data.shape)

# Convert all the values to float 
float_array=np.array(X_train_data,dtype=np.float32)
feature_importance = np.std(float_array, 0)*np.absolute(clf.coef_)

np_column_list = np.array(df_unique.columns.tolist())
column_names = np_column_list[2:45,]

# featureimp_list = feature_importance.split(" ")
# print("List= ", featureimp_list)
print("Column Names=", column_names)
print("Feature importance=", feature_importance)
print(np.sort(feature_importance))

# TimeDiff-X
# StdDev-x
# TimeDiffPeaks-y

Logistic regression Score
91.5129151292
Coefficients of the features
[[ -1.88724066e-03  -9.83008495e-04  -7.51982315e-04   2.61013981e-03
    7.50300714e-03   1.30950985e-02   1.29159338e-02  -2.84148288e-03
   -7.87069654e-03  -4.50040199e-03  -4.05463549e-03  -5.74055765e-03
   -8.46163899e-03  -7.75235496e-03   4.98330745e-03   3.32398493e-03
    6.58256647e-03   9.25132771e-03   1.05576544e-02   5.19746532e-03
   -3.47413495e-03  -1.39415598e-03  -4.31465910e-06   8.36621507e-03
    7.73727324e-03   5.01456960e-03   1.09944892e-04   7.64809839e-04
   -4.40003528e-04  -7.86206795e-04  -6.26431955e-02   5.73220522e-02
    3.79913212e-02  -2.28526452e-01  -1.67523487e-01  -3.30561409e-01
    1.85822547e-02   7.71711888e-02   1.36869849e-01  -2.75264192e-01
   -1.99574649e-01  -3.93126106e-01   1.34086526e-01]]
(415, 43)
Column Names= ['Bin1,x' 'Bin2,x' 'Bin3,x' 'Bin4,x' 'Bin5,x' 'Bin6,x' 'Bin7,x' 'Bin8,x'
 'Bin9,x' 'Bin10,x' 'Bin1,y' 'Bin2,y' 'Bin3,y' 'Bin4,y' 'Bin5,y' 'Bin6,y'
 'Bin

In [35]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, predict, labels=["walking", "running"])
print(cm)

[[145   4]
 [ 19 103]]
